# StyleGAN2-ADA-PyTorch

**Notes**
* Training and Inference sections should be fairly stable. I’ll slowly add new features but it should work for most mainstream use cases.
* Advanced Features are being documented toward the bottom of this notebook

---

If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

## Setup

Let’s start by checking to see what GPU we’ve been assigned. Ideally we get a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-92ca2902-d141-fa4c-dcb0-22792223049b)


Next let’s connect our Google Drive account. This is optional but highly recommended.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt install python3.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib python3.7 python3.7-minimal
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,530 kB of archives.
After this operation, 23.3 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.7-minimal amd64 3.7.16-1+focal1 [588 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.7-minimal amd64 3.7.16-1+focal1 [1,808 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.7-stdlib amd64 3.7.16-1+focal1 [1,773 kB]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.7 amd64 3.

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3
!sudo apt install python3-pip
!sudo apt install python3.7-distutils

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.10   2         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.7    1         manual mode
  3            /usr/bin/python3.8    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/bin/python3.7 to provide /usr/bin/python3 (python3) in manual mode
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-pip python3-setuptools python3-wheel
0 upgraded, 4 newly installed, 0 to remove and 3

In [ ]:
!python --version
!pip --version

Python 3.7.16
pip 20.0.2 from /usr/lib/python3/dist-packages/pip (python 3.7)


## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 142 kB 57.9 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 171 kB 78.5 MB/s 
     |████████████████████████████████| 123 kB 78.9 MB/s 
     |████████████████████████████████| 61 kB 118 kB/s 
     |████████████████████████████████| 156 kB 71.6 MB/s 


/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch


In [ ]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 opensimplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     |████████████████████████████████| 939 kB 29.0 MB/s 
     |████████████████████████████████| 126 kB 74.5 MB/s 
     |████████████████████████████████| 15.7 MB 51.4 MB/s 
     |████████████████████████████████| 65 kB 4.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 175.7 MB 6.5 kB/s 
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088071 sha256=8346c75556f731df1db7efa6bdbd446a82956a7caf88e14d0d57363ccab5b0fd
  Stored in directory: /root/.cache/pip/wheels/1c/05/23/36730377cd7311156f1e5eb5e7c683d5cdac1654658c095cc5
Successfully built jax
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████████

You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch
Already up to date.
Saved working directory and index state WIP on main: 59e05bb added opensimplex


## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

## Train model

In [ ]:
!python stylegan2-ada-pytorch/train.py \
 --outdir={output_folder} \
 --snap={snap} \
 --metrics=none \
 --data={'/content/drive/MyDrive/face_img.zip'} \
 --cfg=auto_norp \
 --cifar_tune=1 \
 --gamma={gamma} \
 --kimg={kimg} \
 --batch={mini_batch} \
 --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume=ffhq_1024 \


python3: can't open file 'stylegan2-ada-pytorch/train.py': [Errno 2] No such file or directory


Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/face_img.zip'
resume_from = '/content/drive/MyDrive/ffhq.pkl'
aug_strength = 0.328
train_count = 10
mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = 'auto'
snapshot_count = 1

In [ ]:
!pip install click requests tqdm pyspng psutil tensorboard ninja imageio-ffmpeg==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 207 kB 41.2 MB/s 
     |████████████████████████████████| 282 kB 64.6 MB/s 
     |████████████████████████████████| 6.0 MB 60.6 MB/s 
     |████████████████████████████████| 26.9 MB 1.2 MB/s 
     |████████████████████████████████| 233 kB 73.8 MB/s 
     |████████████████████████████████| 5.1 MB 35.1 MB/s 
     |████████████████████████████████| 1.0 MB 57.3 MB/s 
     |████████████████████████████████| 781 kB 64.7 MB/s 
     |████████████████████████████████| 181 kB 76.8 MB/s 
     |████████████████████████████████| 93 kB 2.8 MB/s 
     |████████████████████████████████| 4.9 MB 65.0 MB/s 
     |████████████████████████████████| 181 kB 70.7 MB/s 
     |████████████████████████████████| 83 kB 2.8 MB/s 
     |████████████████████████████████| 151 kB 76.7 MB/s 
ERROR: google-auth 2.19.0 has require

In [ ]:
!sudo apt install python3.7-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython3.7 libpython3.7-dev
The following NEW packages will be installed:
  libpython3.7 libpython3.7-dev python3.7-dev
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 5,633 kB of archives.
After this operation, 23.6 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.7 amd64 3.7.16-1+focal1 [1,478 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.7-dev amd64 3.7.16-1+focal1 [3,654 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.7-dev amd64 3.7.16-1+focal1 [501 kB]
Fetched 5,633 kB in 5s (1,179 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results2 --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count --freezed=4


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 1,
  "network_snapshot_ticks": 1,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/face_img.zip",
    "use_labels": false,
    "max_size": 417,
    "xflip": true,
    "resolution": 1024
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {
      "freeze_layers": 10
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
   

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/resized.zip'
resume_from = '/content/drive/MyDrive/ffhq_1024.pkl'
aug_strength = 0.328
train_count = 100
mirror_x = False
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = 'paper256'
snapshot_count = 4

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.